# Seasons Prediction

> **Work done by**: Nwachukwu Anthony  
> **Email**: nwachukwuanthony2015@gmail.com  
> **link to the data set**: https://www.kaggle.com/anthonyemeka12/seasons-images


Telling the Seasons based on Images of sky, trees and houses taken during the various seasons  


### Import Needed Libraries

In [ ]:
import os
from fastai.vision import *

In [ ]:
#Show the folder contents
print(os.listdir("../input/seasonsimages/seasonsImages/autumn"))

In [ ]:
help(untar_data)

In [ ]:
path = Path('../input/seasonsimages/seasonsImages/autumn')

In [ ]:
os.listdir(path)

In [ ]:
for folder in os.listdir("../input/seasonsimages/seasonsImages/"):
    filenames = os.listdir(Path("../input/seasonsimages/seasonsImages/")/folder)
    outputfile = folder+'.csv'
    path = Path("../working/")
    dest=Path("../working/"+folder)
    dest.mkdir(parents=True, exist_ok=True)

    with open(Path("../working/")/folder/outputfile, 'w') as outfile:
        for fname in filenames:
            with open(Path("../input/seasonsimages/seasonsImages/")/folder/fname) as infile:
                for line in infile:
                    outfile.write(line)
    download_images(dest/outputfile, dest, max_pics=200, max_workers=0)
    os.remove(dest/outputfile)
    
if os.path.exists(path/'.ipynb_checkpoints'):
    os.rmdir(path/'.ipynb_checkpoints')
elif os.path.exists(path/'__notebook_source__.ipynb'):
    os.remove(path/'__notebook_source__.ipynb')
else:
    pass

Then we can remove any images that can't be opened:

In [ ]:
classes = os.listdir("../input/seasonsimages/seasonsImages/")
for c in classes:
    print(os.listdir(Path("../working/"+c)))
    verify_images(path/c, delete=True, max_size=500)

In [ ]:
path.ls()

### View

In [ ]:
np.random.seed(42) #makes sure you get same results each time you run the code
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(), size=299, num_workers=4, bs=20).normalize(imagenet_stats)
#NB: <train="."> tells the function that the train set resides in the current folder. This is used 
#when the train and validation sets are in the same folder
#<valid_pct=0.2> takes 20% of the data out for validation

Good! Let's take a look at some of our pictures then.

In [ ]:
#To see the classes, that is the category of data we have, the labels
data.classes

In [ ]:
#Display some of the data
data.show_batch(rows=3,figsize=(7,8))

In [ ]:
#Check the possible labels, the number of training and validation sets
data.classes, data.c, len(data.train_ds), len(data.valid_ds)
#<c> tells how many possible labels

### Train model

In [ ]:
learn.fit_one_cycle(6)

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(15,max_lr=1e-2)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('stage-1')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()
#we seek to find the learning rates.
#learning rate is the steepest downward slope
#if its between 1e-4 and 1e-2, set your learning rate as 3e-5 and 3e-3

In [ ]:
learn.recorder.plot()
# If the plot is not showing try to give a start and end learning rate# learn.lr_find(start_lr=1e-5, end_lr=1e-1)learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2, max_lr=slice(1e-5,1e-4))

In [ ]:
#save the model as 'stage-2'
learn.save('stage-2')

### Interpretation

In [ ]:
#load the mode, "stage-2"
learn.load('stage-2');

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
interp.most_confused(min_val=2)

## Cleaning Up

Some of our top losses aren't due to bad performance by our model. There are images in our data set that shouldn't be.

Using the `ImageCleaner` widget from `fastai.widgets` we can prune our top losses, removing photos that don't belong.

In [ ]:
from fastai.widgets import *

First we need to get the file paths from our top_losses. We can do this with `.from_toplosses`. We then feed the top losses indexes and corresponding dataset to `ImageCleaner`.

Notice that the widget will not delete images directly from disk but it will create a new csv file `cleaned.csv` from where you can create a new ImageDataBunch with the corrected labels to continue training your model.

In [ ]:
db = (ImageList.from_folder(path)
                   .split_none()
                   .label_from_folder()
                   .transform(get_transforms(), size=224)
                   .databunch()
     )

Then we create a new learner to use our new databunch with all the images.

In [ ]:
print(os.listdir('../working/models'))

In [ ]:
learn_cln = cnn_learner(db, models.resnet34, metrics=error_rate)
learn_cln.load('stage-2');
ds, idxs = DatasetFormatter().from_toplosses(learn_cln)
ImageCleaner(ds, idxs, path)

In [ ]:
print(path.ls())

In [ ]:
db = ImageDataBunch.from_csv(path, folder=".", valid_pct=0.2, csv_labels='cleaned.csv',
         ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)

In [ ]:
learn_clnd = cnn_learner(db, models.resnet50, metrics=error_rate)
learn_clnd.lr_find()
learn_clnd.recorder.plot()

In [ ]:
learn_clnd.fit_one_cycle(2, max_lr=slice(1e-4,3e-2))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn_clnd)
interp.plot_confusion_matrix()

In [ ]:
learn_clnd.save('stage-3')

In [ ]:
learn_clnd.unfreeze()
learn_clnd.lr_find()
learn_clnd.recorder.plot()

In [ ]:
learn_clnd.fit_one_cycle(2, max_lr=slice(8e-6,8e-4))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn_clnd)
interp.plot_confusion_matrix()

In [ ]:
learn_clnd.save('stage-4')

In [ ]:
ds, idxs = DatasetFormatter().from_similars(learn_clnd)

In [ ]:
ImageCleaner(ds, idxs, path, duplicates=True)

In [ ]:
print(path.ls())

In [ ]:
db = ImageDataBunch.from_csv(path, folder=".", valid_pct=0.2, csv_labels='cleaned.csv',
         ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)

In [ ]:
learn_clnd = cnn_learner(db, models.resnet50, metrics=error_rate)
learn_clnd.lr_find()
learn_clnd.recorder.plot()

In [ ]:
learn_clnd.fit_one_cycle(2, max_lr=slice(3e-4,3e-2))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn_clnd)
interp.plot_confusion_matrix()

In [ ]:
learn_clnd.save('stage-5')

In [ ]:
learn_clnd.unfreeze()
learn_clnd.lr_find()
learn_clnd.recorder.plot()

In [ ]:
learn_clnd.fit_one_cycle(2, max_lr=slice(3e-5,2e-3))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn_clnd)
interp.plot_confusion_matrix()

In [ ]:
learn_clnd.save('stage-6')

In [ ]:
print(os.listdir('../working/models'))

### Production

First thing first, let's export the content of our `Learner` object for production: Recall that we have **learn** and **learn_clnd**. **learn** contains (stage 1 to 4) while **learn_clnd** contains (stage 5 and 6)

In [ ]:
#export the model 
learn_clnd.export()
#learn.export()
defaults.device = torch.device('cpu')
print(db.classes)

In [ ]:
classe = 'summer'
img = os.listdir(path/classe)[3]
print(img)
img = open_image(path/classe/img)
img

In [ ]:
#make sure that path contains 'export.pkl' by running code below
print(os.listdir(path))

In [ ]:
#load the path
learn = load_learner(path)
#choose the model you wish to use. You can choose between 3 to 6
learn.load('stage-3');
#predict
pred_class,pred_idx,outputs = learn.predict(img)
pred_class,pred_idx,outputs